In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import preprocessing, cross_validation, neighbors, svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier, GradientBoostingClassifier, ExtraTreesClassifier, \
  RandomForestRegressor, AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

from math import sqrt

import pickle
DIR_DATA = "data"
DIR_PROCESSED = "processed"
LABEL = "rating"
NON_PREDICTORS = [LABEL]#, "name","anime_id"]
CV_FOLDS = 5

/home/altock/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# Anime.csv

# anime_id - myanimelist.net's unique id identifying an anime.
# name - full name of anime.
# genre - comma separated list of genres for this anime.
# type - movie, TV, OVA, etc.
# rating - average rating out of 10 for this anime.
# members - number of community members that are in this anime's "group".


# Rating.csv

# user_id - non identifiable randomly generated user id.
# anime_id - the anime that this user has rated.
# rating - rating out of 10 this user has assigned (-1 if the user watched it but didn't assign a rating).

In [5]:
animes = pd.read_csv(DIR_DATA + '/anime.csv')
ratings = pd.read_csv(DIR_DATA + '/rating.csv')

label = animes.rating
labeled_ratings = ratings[ratings.rating != -1][:1000]

# TODO: Get things with similar names classified together

In [4]:
animes.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [5]:
ratings.tail()

,user_id,anime_id,rating
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9
7813736,73516,8074,9


In [6]:
# TODO: Get user_id, previous rankings

In [ ]:
def predict_naive(ratings, animes):
    prediction = []
    for anime_id in ratings.anime_id:
        prediction.append(round(animes[animes.anime_id == anime_id].rating.item()))
    return prediction

In [ ]:
predictions = predict_naive(ratings,animes)

In [7]:
animes[animes.anime_id == 8074].rating

1709    7.46
Name: rating, dtype: float64

In [8]:
print(mean_absolute_error(predictions, ratings.rating.tolist()))
print(mean_squared_error(predictions, ratings.rating.tolist()))

NameError: name 'predictions' is not defined

In [9]:
animes.type = animes.type.astype('category')

In [10]:
animes["genre_list"] = animes["genre"].apply(lambda x: str(x).split(','))

In [11]:
animes[animes.anime_id == 8074].rating

1709    7.46
Name: rating, dtype: float64

In [12]:
print(mean_squared_error(predictions, labeled_ratings.rating.tolist()))

NameError: name 'predictions' is not defined

In [13]:
animes["type"] = animes["type"].astype('category')

In [14]:
animes["genre_list"] = animes["genre"].apply(lambda x: str(x).split(','))

In [15]:
# enc = preprocessing.LabelBinarizer()
# # print(animes.type.cat.codes)
# enc.fit_transform(animes.type.cat.codes)

animes = pd.concat([animes,pd.get_dummies(animes.type)], axis=1).drop(["type"],1)

In [16]:
# pd.get_dummies(animes.genre_list)

In [17]:
animes.head()

,anime_id,name,genre,episodes,rating,members,genre_list,Movie,Music,ONA,OVA,Special,TV
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",1,9.37,200630,"[Drama, Romance, School, Supernatural]",1,0,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",64,9.26,793665,"[Action, Adventure, Drama, Fantasy, Magic,...",0,0,0,0,0,1
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",51,9.25,114262,"[Action, Comedy, Historical, Parody, Samur...",0,0,0,0,0,1
3,9253,Steins;Gate,"Sci-Fi, Thriller",24,9.17,673572,"[Sci-Fi, Thriller]",0,0,0,0,0,1
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",51,9.16,151266,"[Action, Comedy, Historical, Parody, Samur...",0,0,0,0,0,1


In [18]:
g_list = animes.genre_list.tolist()
g_set = set()
for g in g_list:
    for x in g:
        g_str = x.strip()
        if g_str == "Music":
            # Music is also a type, avoid two columns with same name
            g_str = "Music_Genre"
        g_set.add(g_str)
#         print(type(x))

In [19]:
print(sorted(list(g_set)))
pd.get_dummies(list(g_set))

['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music_Genre', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri', 'nan']


,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,nan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [20]:
hot_labels_genres = pd.get_dummies(list(g_set))
def hot_label_sum(genres, hot_labels_genres = hot_labels_genres):
    hot_labels = np.zeros(len(hot_labels_genres["Action"]))
    
    for g in genres:
        g_str = g.strip()
        if g_str == "Music":
          # Music is also a type, avoid two columns with same name
          g_str = "Music_Genre"
        hot_labels += hot_labels_genres[g_str]
    return hot_labels

In [21]:
animes["genre_hot_labels"] = animes.genre_list.apply(lambda x: hot_label_sum(x))

In [22]:
g_df = pd.DataFrame(animes.genre_hot_labels.tolist(),columns=hot_labels_genres.keys())

In [23]:
animes = pd.concat([animes,g_df], axis=1).drop(["genre","genre_list", "genre_hot_labels"],1)

In [24]:
animes.dtypes

anime_id           int64
name              object
episodes          object
rating           float64
members            int64
Movie              uint8
Music              uint8
ONA                uint8
OVA                uint8
Special            uint8
TV                 uint8
Action           float64
Adventure        float64
Cars             float64
Comedy           float64
Dementia         float64
Demons           float64
Drama            float64
Ecchi            float64
Fantasy          float64
Game             float64
Harem            float64
Hentai           float64
Historical       float64
Horror           float64
Josei            float64
Kids             float64
Magic            float64
Martial Arts     float64
Mecha            float64
Military         float64
Music_Genre      float64
Mystery          float64
Parody           float64
Police           float64
Psychological    float64
Romance          float64
Samurai          float64
School           float64
Sci-Fi           float64


In [25]:
# TODO: Episodes
animes.episode = animes.episodes.apply(lambda x: int(x) if x.strip() != "Unknown" else -1)

animes = animes.dropna()

animes["int_rating"] = animes.rating.apply(lambda x: round(x))

In [55]:
# Predict ratings within animes
def predict_on_animes(animes,clf, test_size = 0.2, rating="rating"):
    X = animes.drop(['anime_id','name','rating',"episodes", "int_rating"],1)
    y = animes[rating]  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    
    clf.fit(X_train, y_train)
    
    train_pred = clf.predict(X_train)
    test_pred = clf.predict(X_test)
    
    print("Train RMSE: ", sqrt(mean_squared_error(train_pred, y_train)))
    print("Test RMSE: ", sqrt(mean_squared_error(test_pred, y_test)))
    print()
    print("Train MAE: ", mean_absolute_error(train_pred, y_train))
    print("Test MAE: ", mean_absolute_error(test_pred, y_test))
    
    print()
    print('Training Error: {:.3f}'.format(clf.score(X_train, y_train)))
    print('Testing Error: %.3f' % (clf.score(X_test, y_test)))
    
    print()

In [58]:
# Gradient is best, Random is good too but overfits on training
regressors = [xgb.XGBRegressor(n_estimators=500, nthread=-1)]#,RandomForestRegressor(n_estimators=1000, n_jobs=-1), ]# RandomForestRegressor(), AdaBoostRegressor(), LinearRegression(),svm.SVR()]
for clf in regressors:
    predict_on_animes(animes,clf)
    print()

Train RMSE:  0.5996055600967649
Test RMSE:  0.6482014894627268

Train MAE:  0.441242620508
Test MAE:  0.470201433184

Training Error: 0.660
Testing Error: 0.598




In [54]:
clf = xgb.XGBRegressor(n_estimators=500, nthread=-1)

X = animes.drop(['anime_id','name','rating',"episodes", "int_rating"],1)
y = animes["rating"]  
    
cross_val_score(clf, X, y, cv=CV_FOLDS, n_jobs=-1)

array([-5.5655783 , -6.27749665, -3.80398471, -0.21044408,  0.31508502])

In [59]:
clf = xgb.XGBClassifier(n_estimators=500, nthread=-1)
predict_on_animes(animes, clf, rating="int_rating")
print()


# classifiers = [RandomForestClassifier(n_estimators=500, n_jobs=-1), RandomForestClassifier(n_estimators=500, criterion='entropy', n_jobs=-1), svm.SVC()] #, GradientBoostingClassifier(), AdaBoostClassifier()]
# for clf in classifiers:
#     predict_on_animes(animes,clf, rating="int_rating")
#     print()

Train RMSE:  0.72293854574095
Test RMSE:  0.7993988624178653

Train MAE:  0.399958553518
Test MAE:  0.494819726482

Training Error: 0.654
Testing Error: 0.567




In [53]:
X = animes.drop(['anime_id','name','rating',"episodes", "int_rating"],1)
y = animes["int_rating"]  
    
cross_val_score(clf, X, y, cv=CV_FOLDS, n_jobs=-1)

/home/altock/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


array([ 0.49048013,  0.58036454,  0.59676751,  0.40713101,  0.41178912])

In [ ]:
print(len(list(animes.columns)), len(set(list(animes.columns))))
a_list = list(animes.columns)
for i in set(list(animes.columns)):
    a_list.remove(i)
print(a_list)

In [ ]:
clf = xgb.XGBRegressor()
%timeit clf.fit(X_train, y_train)

In [ ]:
%timeit GradientBoostingRegressor().fit(X_train, y_train)

In [ ]:
animes.head()

In [ ]:
a_file = open(DIR_PROCESSED + '/one_hot_encoded_anime.pickle','wb')
pickle.dump(animes, a_file)
a_file.close()

# Equivalent to above
with open(DIR_PROCESSED + '/one_hot_encoded_anime.pickle','wb') as a_file:
  pickle.dump(animes, a_file)

In [ ]:
x = list(range(100))
y = np.arange(100)
def std_lib(x):
  return sum([i ** 2 for i in x])

def np_lib(x):
  return np.dot(x,x)

In [ ]:
%timeit np_lib(y)

In [ ]:
%timeit std_lib(x)

In [ ]:
print(np_lib(y), std_lib(x))

In [ ]:
np_lib(y)

In [ ]:
std_lib(x)

In [ ]:
x = list(range(5))

In [ ]:
print(x)

In [ ]:
arr = [i**2 for i in x if i != 3]

In [ ]:
arr = []
for i in x:
    if i != 3:
        arr.append(i**2)
print(arr)